# Provided is a notebook that generates a ChatBot focused towards medical questions, followed along from https://colab.research.google.com/drive/1PQXcM_jhN6QJ7uTkxvNbxoI54r03uSr3?usp=sharing from Irina Nik's video

In [11]:
!pip install llama-index==0.5.6
!pip install langchain==0.0.148

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define prompt helper
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=1.5, model_name="text-davinci-003", max_tokens=num_outputs))

    documents = SimpleDirectoryReader(directory_path).load_data()

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

    index.save_to_disk('index.json')

    return index

def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True:
        query = input("What do you want to ask? ")
        response = index.query(query)
        display(Markdown(f"<b>{response.response}</b>"))

In [ ]:
os.environ["OPENAI_API_KEY"] = input("Enter OpenAI key:")

OpenAI's embedding API will now embed all given entries in 'example_data_folder'.

In [14]:
construct_index("./example_data_folder")

In [ ]:
ask_ai()

What do you want to ask? blood pressure


<b>
Answer: Increased heart rate</b>

What do you want to ask? The urine of a patient is red. What does this mean?


<b>
Answer: This could indicate the presence of blood in the urine, which could be a sign of a urinary tract infection, kidney infection, or other medical condition such as anemia, which is a common symptom of depression.</b>